In [1]:
print('hi')

hi


In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings

import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import CTransformers


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [7]:
index_name = 'medical-chatbot'

In [9]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data, glob='*.pdf', loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents
